 # Homework 5 

 ### 2 . CIFAR10 - Classification 

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

Feature structure: 
id: Text(dtype=object)
image: Image(shape=(32,32,3), dtype=uint8)
label: ClassLabel(dtype=int64, num_classes=10)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
train_ds, test_ds = tfds.load('cifar-10', split=['train', 'test'], as_supervised=True)